In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

## Let us import all the models

In [2]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
df = pd.read_csv(r"stud.csv")

In [4]:
df

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


## Let us drop math score, and try to predict with our dependednt an dindependednt features..

In [5]:
X = df.drop(columns = ["math_score"], axis =1)

In [6]:
Y = df.math_score

In [7]:
X.head(3)

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93


## Let us dive into Columntransformer to work with our numerical and categorical features

In [8]:
numerical_features = X.select_dtypes(exclude = "object").columns
categorical_features = X.select_dtypes(include = "object").columns

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [10]:
tf = ColumnTransformer(
    [("Onehot", OneHotEncoder(), categorical_features),
    ("StandScale", StandardScaler(), numerical_features)
    ]    
)

In [11]:
X = tf.fit_transform(X)

In [12]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [13]:
Y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

## Let us Train and split em'

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 2 )

In [16]:
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(800, 19) (800,)
(200, 19) (200,)


## Deep into all the models, train, and evaluate them...

In [17]:
'''
linear = LinearRegression()
linear.fit(X_train, Y_train)
y_pred = linear.predict(X_test)
r2_square = r2_score(Y_test, y_pred)
r2_square
'''

'\nlinear = LinearRegression()\nlinear.fit(X_train, Y_train)\ny_pred = linear.predict(X_test)\nr2_square = r2_score(Y_test, y_pred)\nr2_square\n'

In [18]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [19]:
ML_Models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

## Code to apply every model to get evaluated

In [24]:
model = list(ML_Models.values())[0]
model

LinearRegression()

In [41]:
for i in range(len(list(ML_Models))):

    model = list(ML_Models.values())[i]
    #obj = model
    model.fit(X_train,Y_train)
    
    train_pred = obj.predict(X_train)
    train_mae ,train_rmse, train_r2 = evaluate_model(Y_train, train_pred)
    print(list(ML_Models.keys())[i])
    print("\n")
    
    print("Model performance for Training set")
    print("Root Mean Squared Error:", train_mae)
    print("Mean Absolute Error:", train_rmse)
    print("R2 Score:",train_r2 )

    print('----------------------------------')

    test_pred = model.predict(X_test)
    test_mae ,test_rmse, test_r2 = evaluate_model(Y_test,test_pred)

    print("Model performance for test set")
    print("Root Mean Squared Error:", test_mae)
    print("Mean Absolute Error:", test_rmse)
    print("R2 Score:",test_r2 )

    print("\n")
    

Linear Regression


Model performance for Training set
Root Mean Squared Error: 4.6912731912985075
Mean Absolute Error: 5.785107939013268
R2 Score: 0.8554566388016751
----------------------------------
Model performance for test set
Root Mean Squared Error: 4.4840625
Mean Absolute Error: 5.735973891535334
R2 Score: 0.8519273093661903


Lasso


Model performance for Training set
Root Mean Squared Error: 4.6912731912985075
Mean Absolute Error: 5.785107939013268
R2 Score: 0.8554566388016751
----------------------------------
Model performance for test set
Root Mean Squared Error: 5.5796150146366825
Mean Absolute Error: 6.936635538475354
R2 Score: 0.7834498995774966


Ridge


Model performance for Training set
Root Mean Squared Error: 4.6912731912985075
Mean Absolute Error: 5.785107939013268
R2 Score: 0.8554566388016751
----------------------------------
Model performance for test set
Root Mean Squared Error: 4.463040146647811
Mean Absolute Error: 5.7124438550463035
R2 Score: 0.85313966128

In [38]:
model_list = []
r2_list =[]


for i in range(len(list(ML_Models))):
    model = list(ML_Models.values())[i]
    model.fit(X_train, Y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(Y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(Y_test, y_test_pred)

    
    print(list(ML_Models.keys())[i])
    model_list.append(list(ML_Models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.2409
- Mean Absolute Error: 4.1980
- R2 Score: 0.8814
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.7360
- Mean Absolute Error: 4.4841
- R2 Score: 0.8519


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.4732
- Mean Absolute Error: 5.0840
- R2 Score: 0.8190
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.9366
- Mean Absolute Error: 5.5796
- R2 Score: 0.7834


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.2362
- Mean Absolute Error: 4.1929
- R2 Score: 0.8816
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.7124
- Mean Absolute Error: 4.4630
- R2 Score: 0.8531


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 5.6846
- Mean Absolute Error: 4.5500
- R2 Score: 0.8604
-----------------------

In [42]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
2,Ridge,0.853140
0,Linear Regression,0.851927
7,CatBoosting Regressor,0.830806
5,Random Forest Regressor,0.813258
6,XGBRegressor,0.813056
8,AdaBoost Regressor,0.797917
1,Lasso,0.783450
3,K-Neighbors Regressor,0.732665
4,Decision Tree,0.698748
